# LCO study speed 115 m/s

This is an example using SHARPy




## Required Packages

In [1]:
# Loading of the used packages
import numpy as np              # basic mathematical and array functions
import os                       # Functions related to the operating system
import matplotlib.pyplot as plt # Plotting library

import sharpy.sharpy_main                  # Run SHARPy inside jupyter notebooks
import sharpy.utils.plotutils as pu        # Plotting utilities
from sharpy.utils.constants import deg2rad # Constant to conver degrees to radians

import sharpy.utils.generate_cases as gc

## Problem Set-up

### Velocity

The UVLM is assembled in normalised time at a velocity of $1 m/s$. The only matrices that need updating then with free stream velocity are the structural matrices, which is significantly cheaper to do than to update the UVLM.

In [2]:
! pip install plotly

In [3]:
# Geometry
chord = 1.         # Chord of the wing
aspect_ratio = 16. # Ratio between lenght and chord: aspect_ratio = length/chord
wake_length = 25   # Length of the wake in chord lengths

# Discretization
num_node = 21           # Number of nodes in the structural discretisation
                        # The number of nodes will also define the aerodynamic panels in the
                        # spanwise direction
num_chord_panels = 4    # Number of aerodynamic panels in the chordwise direction
num_points_camber = 100 # The camber line of the wing will be defined by a series of (x,y)
                        # coordintes. Here, we define the size of the (x,y) vectors

# Structural properties of the beam cross section
mass_per_unit_length = 0.75 # Mass per unit length
mass_iner_x = 0.1           # Mass inertia around the local x axis
mass_iner_y = 0.05          # Mass inertia around the local y axis
mass_iner_z = 0.05          # Mass inertia around the local z axis
pos_cg_B = np.zeros((3))    # position of the centre of mass with respect to the elastic axis
EA = 1e7                    # Axial stiffness
GAy = 1e6                   # Shear stiffness in the local y axis
GAz = 1e6                   # Shear stiffness in the local z axis
GJ = 3e5                    # Torsional stiffness
EIy = 6e5                   # Bending stiffness around the flapwise direction
EIz = 1.5e8                   # Bending stiffness around the edgewise direction

# Operation
WSP = 115.             # Wind speed
air_density = 0.1      # Air density

# Time discretization
end_time = 10.0                  # End time of the simulation
dt = 2*chord/num_chord_panels/WSP # Always keep one timestep per panel



In [4]:
aoa_ini_deg = 4.        # Angle of attack at the beginning of the simulation
aoa_end_deg = 1.        # Angle of attack at the end of the simulation

In [5]:
wing = gc.AeroelasticInformation()

In [6]:
wing.StructuralInformation.__dict__.keys()

dict_keys(['num_node_elem', 'num_node', 'num_elem', 'coordinates', 'connectivities', 'elem_stiffness', 'stiffness_db', 'elem_mass', 'mass_db', 'frame_of_reference_delta', 'structural_twist', 'boundary_conditions', 'beam_number', 'body_number', 'app_forces', 'lumped_mass_nodes', 'lumped_mass', 'lumped_mass_inertia', 'lumped_mass_position', 'lumped_mass_mat', 'lumped_mass_mat_nodes'])

In [7]:
# Define the number of nodes and the number of nodes per element
wing.StructuralInformation.num_node = num_node
wing.StructuralInformation.num_node_elem = 3
# Compute the number of elements assuming basic connections
wing.StructuralInformation.compute_basic_num_elem()

In [8]:
# Generate an array with the location of the nodes
node_r = np.zeros((num_node, 3))
node_r[:,1] = np.linspace(0, chord*aspect_ratio, num_node)

In [9]:
wing.StructuralInformation.generate_uniform_beam(node_r,
                    mass_per_unit_length,
                    mass_iner_x,
                    mass_iner_y,
                    mass_iner_z,
                    pos_cg_B,
                    EA,
                    GAy,
                    GAz,
                    GJ,
                    EIy,
                    EIz,
                    num_node_elem = wing.StructuralInformation.num_node_elem,
                    y_BFoR = 'x_AFoR',
                    num_lumped_mass=0)

In [10]:
# print(wing.StructuralInformation.connectivities)

In [11]:
wing.StructuralInformation.boundary_conditions[0] = 1
wing.StructuralInformation.boundary_conditions[-1] = -1

In [12]:
# Compute the number of panels in the wake (streamwise direction) based on the previous paramete
wake_panels = int(wake_length*chord/dt)

# Define the coordinates of the camber line of the wing
wing_camber = np.zeros((1, num_points_camber, 2))
wing_camber[0, :, 0] = np.linspace(0, 1, num_points_camber)



In [13]:
# Generate blade aerodynamics
wing.AerodynamicInformation.create_one_uniform_aerodynamics(wing.StructuralInformation,
                                 chord = chord,
                                 twist = 0.,
                                 sweep = 0.,
                                 num_chord_panels = num_chord_panels,
                                 m_distribution = 'uniform',
                                 elastic_axis = 0.5,
                                 num_points_camber = num_points_camber,
                                 airfoil = wing_camber)

In [14]:
# Gather data about available solvers
SimInfo = gc.SimulationInformation() # Initialises the SimulationInformation class
SimInfo.set_default_values()         # Assigns the default values to all the solvers

# Print the available solvers and postprocessors
# for key in SimInfo.solvers.keys():
#     print(key)



In [15]:
SimInfo.solvers['BeamLoader']



{'unsteady': True, 'orientation': [1.0, 0, 0, 0], 'for_pos': [0.0, 0, 0]}

In [16]:
SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                        'AerogridLoader']

SimInfo.solvers['SHARPy']['case'] = 'plot'
SimInfo.solvers['SHARPy']['route'] = './'
SimInfo.solvers['SHARPy']['write_screen'] = 'off'



In [17]:
SimInfo.solvers['AerogridLoader']['unsteady'] = 'on'
SimInfo.solvers['AerogridLoader']['mstar'] = wake_panels
SimInfo.solvers['AerogridLoader']['freestream_dir'] = np.array([1.,0.,0.])
SimInfo.solvers['AerogridLoader']['wake_shape_generator'] = 'StraightWake'
SimInfo.solvers['AerogridLoader']['wake_shape_generator_input'] = {'u_inf': WSP,
                                                                   'u_inf_direction' : np.array(
                                                                                         [np.cos(aoa_ini_deg*deg2rad),
                                                                                         0.,
                                                                                         np.sin(aoa_ini_deg*deg2rad)]),
                                                                   'dt': dt}



In [18]:
gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
wing.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
SimInfo.generate_solver_file()

In [19]:
sharpy_output = sharpy.sharpy_main.main(['',
                                         SimInfo.solvers['SHARPy']['route'] +
                                         SimInfo.solvers['SHARPy']['case'] +
                                         '.sharpy'])



fatal: not a git repository (or any of the parent directories): .git


In [20]:
#pu.plot_timestep(sharpy_output, tstep=-1, minus_mstar=(wake_panels - 6), plotly=True)

In [21]:
# Define the simulation
SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                        'AerogridLoader',
                        'StaticCoupled']

SimInfo.set_variable_all_dicts('rho', air_density)

SimInfo.solvers['SHARPy']['case'] = 'static'
SimInfo.solvers['SHARPy']['write_screen'] = 'on'

SimInfo.solvers['NonLinearStatic']['gravity_on'] = False

SimInfo.solvers['StaticUvlm']['horseshoe'] = True
SimInfo.solvers['StaticUvlm']['n_rollup'] = 0
SimInfo.solvers['StaticUvlm']['velocity_field_generator'] = 'SteadyVelocityField'
SimInfo.solvers['StaticUvlm']['velocity_field_input'] = {'u_inf' : WSP,
                                                         'u_inf_direction' : np.array(
                                                                                [np.cos(aoa_ini_deg*deg2rad),
                                                                                 0.,
                                                                                 np.sin(aoa_ini_deg*deg2rad)])}

SimInfo.solvers['StaticCoupled']['structural_solver'] = 'NonLinearStatic'
SimInfo.solvers['StaticCoupled']['structural_solver_settings'] = SimInfo.solvers['NonLinearStatic']
SimInfo.solvers['StaticCoupled']['aero_solver'] = 'StaticUvlm'
SimInfo.solvers['StaticCoupled']['aero_solver_settings'] = SimInfo.solvers['StaticUvlm']
SimInfo.solvers['StaticCoupled']['n_load_steps'] = 0


In [22]:
gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
SimInfo.generate_solver_file()
wing.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])

In [23]:
# Running SHARPy again inside jupyter
sharpy_output = sharpy.sharpy_main.main(['',
                                         SimInfo.solvers['SHARPy']['route'] +
                                         SimInfo.solvers['SHARPy']['case'] +
                                         '.sharpy'])

--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
            ######  ##     ## ##     ## ##     ## ##           ##
--------------------------------------------------------------------------------
Aeroelastics Lab, Aeronautics Department.
    Copyright (c), Imperial College London.
    All rights reserved.
    License available at https://github.com/imperialcollegelondon/sharpy
Running SHARPy from /home/jesusgp/sharpy/docs/source/content/example_notebooks/HALE_wing_time_marching/Speed_115
SHARPy being run is in /home/jesusgp/anaconda3/envs/sharpy_env/lib/python

fatal: not a git repository (or any of the parent directories): .git


Generating an instance of StaticCoupled
Generating an instance of NonLinearStatic
Generating an instance of StaticUvlm



|=====|=====|============|==========|==========|==========|==========|==========|==========|
|iter |step | log10(res) |    Fx    |    Fy    |    Fz    |    Mx    |    My    |    Mz    |
|=====|=====|============|==========|==========|==========|==========|==========|==========|
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
|  0  |  0  |  0.00000   |-183.7072 |-835.6037 |3884.3698 |31683.7231| 735.0058 |1529.1149 |
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      R

In [24]:
# pu.plot_timestep(sharpy_output, tstep=-1, minus_mstar=(wake_panels-6), plotly=True)


In [25]:
SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                        'AerogridLoader',
                        'StaticCoupled',
                        'DynamicCoupled']

SimInfo.solvers['SHARPy']['route'] = './'
SimInfo.solvers['SHARPy']['case'] = 'dynamic'

In [26]:
# Compute the number of time steps needed based on the previous parameters
time_steps = int(end_time/dt)

# Define the time step and the number of time steps in every solver that requires them as input
SimInfo.set_variable_all_dicts('dt', dt)
SimInfo.define_num_steps(time_steps)

In [27]:
SimInfo.solvers['StepUvlm']['convection_scheme'] = 2
SimInfo.solvers['StaticUvlm']['velocity_field_generator'] = 'SteadyVelocityField'
SimInfo.solvers['StepUvlm']['velocity_field_input'] = {'u_inf' : WSP,
                                                       'u_inf_direction' : np.array(
                                                                              [np.cos(aoa_end_deg*deg2rad),
                                                                               0.,
                                                                               np.sin(aoa_end_deg*deg2rad)])}

SimInfo.solvers['NonLinearDynamicPrescribedStep']['gravity_on'] = False

SimInfo.solvers['DynamicCoupled']['structural_solver'] = 'NonLinearDynamicPrescribedStep'
SimInfo.solvers['DynamicCoupled']['structural_solver_settings'] = SimInfo.solvers['NonLinearDynamicPrescribedStep']
SimInfo.solvers['DynamicCoupled']['aero_solver'] = 'StepUvlm'
SimInfo.solvers['DynamicCoupled']['aero_solver_settings'] = SimInfo.solvers['StepUvlm']
SimInfo.solvers['DynamicCoupled']['postprocessors'] = ['BeamPlot', 'AerogridPlot']
SimInfo.solvers['DynamicCoupled']['postprocessors_settings'] = {'BeamPlot': SimInfo.solvers['BeamPlot'],
                                                             'AerogridPlot': SimInfo.solvers['AerogridPlot']}


In [28]:
SimInfo.with_forced_vel = True
SimInfo.for_vel = np.zeros((time_steps,6), dtype=float)
SimInfo.for_acc = np.zeros((time_steps,6), dtype=float)
SimInfo.with_dynamic_forces = True
SimInfo.dynamic_forces = np.zeros((time_steps,wing.StructuralInformation.num_node,6),
                                  dtype=float)



In [29]:
gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
wing.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
SimInfo.generate_solver_file()
SimInfo.generate_dyn_file(time_steps)

In [ ]:
sharpy_output = sharpy.sharpy_main.main(['',
                                         SimInfo.solvers['SHARPy']['route'] +
                                         SimInfo.solvers['SHARPy']['case'] +
                                         '.sharpy'])

--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
            ######  ##     ## ##     ## ##     ## ##           ##
--------------------------------------------------------------------------------
Aeroelastics Lab, Aeronautics Department.
    Copyright (c), Imperial College London.
    All rights reserved.
    License available at https://github.com/imperialcollegelondon/sharpy
Running SHARPy from /home/jesusgp/sharpy/docs/source/content/example_notebooks/HALE_wing_time_marching/Speed_115
SHARPy being run is in /home/jesusgp/anaconda3/envs/sharpy_env/lib/python

fatal: not a git repository (or any of the parent directories): .git


Generating an instance of StaticCoupled
Generating an instance of NonLinearStatic
Generating an instance of StaticUvlm



|=====|=====|============|==========|==========|==========|==========|==========|==========|
|iter |step | log10(res) |    Fx    |    Fy    |    Fz    |    Mx    |    My    |    Mz    |
|=====|=====|============|==========|==========|==========|==========|==========|==========|
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
|  0  |  0  |  0.00000   |-183.7072 |-835.6037 |3884.3698 |31683.7231| 735.0058 |1529.1149 |
                      DeltaF      DeltaX         Res      ResRel      ResFrc   ResRelFrc      ResMmt   ResRelMmt         ErX       ErPos       ErPsi
LoadStep Subiter      DeltaF     DeltaX          Res      R

|  55   | 0.2391 |  4   |   0.007095   |   7.436302   |  -5.112956   | 0.000000e+00 | 0.000000e+00 |
|  56   | 0.2435 |  5   |   0.007131   |   8.943248   |  -5.305727   | 0.000000e+00 | 0.000000e+00 |
|  57   | 0.2478 |  5   |   0.007096   |   8.954147   |  -5.242098   | 0.000000e+00 | 0.000000e+00 |
|  58   | 0.2522 |  5   |   0.007141   |   8.932123   |  -5.497017   | 0.000000e+00 | 0.000000e+00 |
|  59   | 0.2565 |  5   |   0.007134   |   8.947936   |  -5.362098   | 0.000000e+00 | 0.000000e+00 |
|  60   | 0.2609 |  6   |   0.007072   |  10.386043   |  -5.776408   | 0.000000e+00 | 0.000000e+00 |
|  61   | 0.2652 |  6   |   0.007098   |  10.414675   |  -5.579622   | 0.000000e+00 | 0.000000e+00 |
|  62   | 0.2696 |  6   |   0.007098   |  10.437697   |  -5.563443   | 0.000000e+00 | 0.000000e+00 |
|  63   | 0.2739 |  5   |   0.007112   |   8.892268   |  -5.063506   | 0.000000e+00 | 0.000000e+00 |
|  64   | 0.2783 |  5   |   0.007131   |   8.907840   |  -5.576788   | 0.000000e+00 | 0.000

|  134  | 0.5826 |  6   |   0.007089   |  10.415112   |  -5.694527   | 0.000000e+00 | 0.000000e+00 |
|  135  | 0.5870 |  6   |   0.012788   |  10.497664   |  -5.616430   | 0.000000e+00 | 0.000000e+00 |
|  136  | 0.5913 |  6   |   0.007040   |  10.439010   |  -5.686140   | 0.000000e+00 | 0.000000e+00 |
|  137  | 0.5957 |  5   |   0.007114   |   8.933511   |  -5.529012   | 0.000000e+00 | 0.000000e+00 |
|  138  | 0.6000 |  6   |   0.007096   |  10.419116   |  -5.679481   | 0.000000e+00 | 0.000000e+00 |
|  139  | 0.6043 |  6   |   0.007083   |  10.424538   |  -5.425286   | 0.000000e+00 | 0.000000e+00 |
|  140  | 0.6087 |  6   |   0.007130   |  10.395191   |  -5.415100   | 0.000000e+00 | 0.000000e+00 |
|  141  | 0.6130 |  6   |   0.007100   |  10.414209   |  -5.543597   | 0.000000e+00 | 0.000000e+00 |
|  142  | 0.6174 |  6   |   0.007112   |  10.424595   |  -5.757619   | 0.000000e+00 | 0.000000e+00 |
|  143  | 0.6217 |  5   |   0.007095   |   8.944633   |  -5.070077   | 0.000000e+00 | 0.000

|  213  | 0.9261 |  6   |   0.007083   |  10.421754   |  -5.447666   | 0.000000e+00 | 0.000000e+00 |
|  214  | 0.9304 |  6   |   0.007053   |  10.426520   |  -5.368278   | 0.000000e+00 | 0.000000e+00 |
|  215  | 0.9348 |  6   |   0.007066   |  10.546547   |  -5.358781   | 0.000000e+00 | 0.000000e+00 |
|  216  | 0.9391 |  6   |   0.007204   |  10.449870   |  -5.411589   | 0.000000e+00 | 0.000000e+00 |
|  217  | 0.9435 |  6   |   0.007074   |  10.440827   |  -5.469148   | 0.000000e+00 | 0.000000e+00 |
|  218  | 0.9478 |  6   |   0.007080   |  10.429012   |  -5.485996   | 0.000000e+00 | 0.000000e+00 |
|  219  | 0.9522 |  6   |   0.007098   |  10.462953   |  -5.516029   | 0.000000e+00 | 0.000000e+00 |
|  220  | 0.9565 |  5   |   0.006990   |   9.093015   |  -5.067278   | 0.000000e+00 | 0.000000e+00 |
|  221  | 0.9609 |  5   |   0.007093   |   8.994424   |  -5.058577   | 0.000000e+00 | 0.000000e+00 |
|  222  | 0.9652 |  6   |   0.007106   |  10.490829   |  -5.577695   | 0.000000e+00 | 0.000

|  292  | 1.2696 |  6   |   0.007047   |  10.791723   |  -5.421321   | 0.000000e+00 | 0.000000e+00 |
|  293  | 1.2739 |  6   |   0.007004   |  10.720311   |  -5.408370   | 0.000000e+00 | 0.000000e+00 |
|  294  | 1.2783 |  6   |   0.007058   |  10.852892   |  -5.416937   | 0.000000e+00 | 0.000000e+00 |
|  295  | 1.2826 |  6   |   0.006745   |  11.166439   |  -5.455957   | 0.000000e+00 | 0.000000e+00 |
|  296  | 1.2870 |  6   |   0.007051   |  10.762275   |  -5.472477   | 0.000000e+00 | 0.000000e+00 |
|  297  | 1.2913 |  6   |   0.006948   |  10.824673   |  -5.447575   | 0.000000e+00 | 0.000000e+00 |
|  298  | 1.2957 |  6   |   0.006987   |  10.762703   |  -5.440414   | 0.000000e+00 | 0.000000e+00 |
|  299  | 1.3000 |  6   |   0.006540   |  11.490926   |  -5.628604   | 0.000000e+00 | 0.000000e+00 |
|  300  | 1.3043 |  6   |   0.006715   |  11.237258   |  -5.675421   | 0.000000e+00 | 0.000000e+00 |
|  301  | 1.3087 |  6   |   0.006729   |  11.109321   |  -5.442969   | 0.000000e+00 | 0.000

In [ ]:
pu.plot_timestep(sharpy_output, tstep=-400, minus_mstar=(wake_panels-6), plotly=True)

In [ ]:
time = np.linspace(0, dt*time_steps, time_steps)
tip_pos = np.zeros((time_steps))
for it in range(time_steps):
    tip_pos[it] = sharpy_output.structure.timestep_info[it].pos[-1, 2]

fig, plots = plt.subplots(1, 1, figsize=(6, 3))

plots.grid()
plots.set_xlabel("time [s]")
plots.set_ylabel("tip position [m]")
plots.plot(time, tip_pos, '-')

plt.show()



In [ ]:
print(sharpy_output.structure.timestep_info)